In [1]:
import re
import numpy as np
import torch
from torch import nn
import math
import torch.utils.data as data
import torch.optim as optim
import time

In [10]:
def read_file(filename):
    list_x = []
    list_y = []
    ins = open(filename, "r")
    for line in ins:
        cache = line.rstrip('\n')
        cache = cache.replace("A", "1")
        cache = cache.replace("C", "2")
        cache = cache.replace("G", "3")
        cache = cache.replace("T", "4")
        cache = cache.replace("N", "5")
        list_x.append(cache.split('\t')[0])
        list_y.append(cache.split('\t')[1])
    x = np.array(list_x)
    y = np.array(list_y)
    x = x.reshape(len(x), 1)
    y = y.reshape(len(y), 1)
    return x, y


def read_file_105_110(filename):  # 只读取105_110长度的序列存入矩阵
    list_x = []
    list_y = []
    ins = open(filename, "r")
    for line in ins:
        cache = line.rstrip('\n')
        cache = cache.replace("A", "1")
        cache = cache.replace("C", "2")
        cache = cache.replace("G", "3")
        cache = cache.replace("T", "4")
        cache = cache.replace("N", "5")
        if 104 < len(cache.split('\t')[0]) < 111:
            list_x.append(cache.split('\t')[0])
            list_y.append(cache.split('\t')[1])
    x = np.array(list_x)
    y = np.array(list_y)
    x = x.reshape(len(x), 1)
    y = y.reshape(len(y), 1)
    return x, y


def read_file_105_110_withoutN(filename):  # 只读取105_110长度的序列存入矩阵,有N就去掉
    list_x = []
    list_y = []
    ins = open(filename, "r")
    for line in ins:
        cache = line.rstrip('\n')
        cache = cache.replace("A", "1")
        cache = cache.replace("C", "2")
        cache = cache.replace("G", "3")
        cache = cache.replace("T", "4")
        if 104 < len(cache.split('\t')[0]) < 111:
            if not ("N" in cache.split('\t')[0]):
                list_x.append(cache.split('\t')[0])
                list_y.append(cache.split('\t')[1])
    x = np.array(list_x)
    y = np.array(list_y)
    x = x.reshape(len(x), 1)
    y = y.reshape(len(y), 1)
    return x, y

def read_file_110_withoutN(filename):  # 只读取105_110长度的序列存入矩阵,有N就去掉
    list_x = []
    list_y = []
    cache_x = []
    ins = open(filename, "r")
    for line in ins:
        cache = line.rstrip('\n')
        cache = cache.replace("A", "1")
        cache = cache.replace("C", "2")
        cache = cache.replace("G", "3")
        cache = cache.replace("T", "4")
        if len(cache.split('\t')[0]) ==110:
            if not ("N" in cache.split('\t')[0]):
                cache_x.append(cache.split('\t')[1])
                a=cache_x[len(cache_x)-1]
                if (float(a)==int(float(a))):
                    list_x.append(cache.split('\t')[0])
                    list_y.append(cache.split('\t')[1])
    x = np.array(list_x)
    y = np.array(list_y)
    x = x.reshape(len(x), 1)
    y = y.reshape(len(y), 1)
    return x, y

def read_file_test(filename):  # 只读取105_110长度的序列存入矩阵,有N就去掉
    list_x = []
    list_y = []
    seq = []
    ins = open(filename, "r")
    for line in ins:
        cache = line.rstrip('\n')
        seq.append(cache.split('\t')[0])
        cache = cache.replace("A", "1")
        cache = cache.replace("C", "2")
        cache = cache.replace("G", "3")
        cache = cache.replace("T", "4")
        list_x.append(cache.split('\t')[0])
        list_y.append(cache.split('\t')[1])
    x = np.array(list_x)
    y = np.array(list_y)
    x = x.reshape(len(x), 1)
    y = y.reshape(len(y), 1)
    return x, y,seq

def max_len_matrix(matrix):  # 矩阵大小应为n*1，返回最长序列的长度
    num_max = 0
    for i in range(len(matrix)):
        a = matrix[i, 0]
        if len(a) > num_max:
            num_max = len(a)
    return num_max


# num_max,num=max_len_matrix(train_x)
# out:142，大于110序列的有41136个
# print(num_max,num)
# print(train_x.shape)
def info_len_matrix(matrix):  # 矩阵大小应为n*1，返回序列的长度信息
    num_min = 143
    num_110 = 0
    num_105 = 0
    for i in range(len(matrix)):
        a = matrix[i, 0]
        if len(a) < 110:
            num_110 = num_110 + 1
        if len(a) < 105:
            num_105 = num_105 + 1
        if len(a) < num_min:
            num_min = len(a)
    print("最小长度", num_min)  # output:78
    print("小于110序列长度的有", num_110)  # output:1168938 小于108:152750 小于109:319525
    print("小于105序列长度的有", num_105)  # 小于105:62448 小于104:51072 小于100:26142
    return num_min


def padding_x(matrix, num_max):  # 矩阵大小应为n*1,num_max为现有数据最大长度，将长度小于最长序列的序列补0到最长序列
    for i in range(len(matrix)):
        a = matrix[i, 0]
        if len(a) < num_max:
            for j in range(num_max - len(a)):
                matrix[i, 0] = matrix[i, 0] + "0"
    return matrix


def expend_matrix(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[0]) - 1):
            matrix[i][j + 1] = matrix[i, 0][j]
    return matrix


def process_data_max142(filename):
    x, y = read_file(filename)
    max_num = max_len_matrix(x)
    x = padding_x(x, 142)  # 现在序列长度全都是142,不到142的用0扩充到142

    # 现在有六百万条数据，每个数据有142个序列。现在将每个数据的序列展开，方便以后调用。shape为6739258, 143
    # 本来一条序列有142，多出来的一是原来的数据是原来的序列：6739258，0.
    x = np.hstack((x, np.ones((len(x), max_num))))  # 扩展矩阵

    x = expend_matrix(x)  # 向扩展后的矩阵写入数据，即将每一条序列展开
    x = x.astype(np.float64)  # 转化成数字方便计算
    y = y.astype(np.float64)  # 转化成数字方便计算
    x = np.delete(x, 0, axis=1)  # 删掉原来的长序列，现在矩阵中一行就是一个序列，一行有142个列
    return x, y


def process_data_105_110(filename):  # 序列长度只要105_110
    x, y = read_file_105_110(filename)
    x = padding_x(x, 110)  # 现在序列长度全都是110,不到110的用0扩充到110
    x = np.hstack((x, np.ones((len(x), 110))))  # 扩展矩阵
    x = expend_matrix(x)  # 向扩展后的矩阵写入数据，即将每一条序列展开
    x = x.astype(np.float64)  # 转化成数字方便计算
    y = y.astype(np.float64)  # 转化成数字方便计算
    x = np.delete(x, 0, axis=1)  # 删掉原来的长序列，现在矩阵中一行就是一个序列，一行有142个列
    return x, y


def process_data_110_withoutN(filename):  # 序列长度只要105_110没有N
    # 因为有N的序列 最小长度 78
    # 小于110序列长度的有 5745，即有N的序列只有五千条
    # 小于105序列长度的有 188
    # 所以决定先去掉N
    x, y = read_file_110_withoutN(filename)
    x = padding_x(x, 110)  # 现在序列长度全都是110,不到110的用0扩充到110
    x = np.hstack((x, np.ones((len(x), 110))))  # 扩展矩阵
    x = expend_matrix(x)  # 向扩展后的矩阵写入数据，即将每一条序列展开
    x = x.astype(np.float64)  # 转化成数字方便计算
    y = y.astype(np.float64)  # 转化成数字方便计算
    x = np.delete(x, 0, axis=1)  # 删掉原来的长序列，现在矩阵中一行就是一个序列，一行有142个列
    return x, y

def process_data_test(filename):  # 序列长度只要105_110没有N
    # 因为有N的序列 最小长度 78
    # 小于110序列长度的有 5745，即有N的序列只有五千条
    # 小于105序列长度的有 188
    # 所以决定先去掉N
    x, y,seq = read_file_test(filename)
    x = np.hstack((x, np.ones((len(x), 110))))  # 扩展矩阵
    x = expend_matrix(x)  # 向扩展后的矩阵写入数据，即将每一条序列展开
    x = x.astype(np.float64)  # 转化成数字方便计算
    y = y.astype(np.float64)  # 转化成数字方便计算
    x = np.delete(x, 0, axis=1)  # 删掉原来的长序列，现在矩阵中一行就是一个序列，一行有142个列
    return x,y,seq


In [3]:
class PositionalEncoding(nn.Module):
    """位置编码"""

    def __init__(self, num_hiddens, dropout, max_len=110):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 创建一个足够长的P
        self.P = torch.zeros((1, max_len, num_hiddens))
        x = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1) / torch.pow(10000, torch.arange(0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        self.P[:, :, 0::2] = torch.sin(x)
        self.P[:, :, 1::2] = torch.cos(x)

    def forward(self, x):
        x = x + self.P[:, :x.shape[1], :].to(device)
        return self.dropout(x)


class Net(nn.Module):
    def __init__(self, num_hiddens, sequence_type, max_len, multi_hiddens, num_heads,
                 num_layers, ffn_num_input, ffn_num_hiddens, norm1, norm2, dropout,number_of_categories, **kwargs):
        super(Net, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.max_len = max_len
        self.embedding = nn.Embedding(sequence_type, num_hiddens) # embedding
        self.pos_encoding = PositionalEncoding(num_hiddens, dropout, max_len)  # 位置编码

        self.blks = nn.Sequential()  # 定义数个encoder
        for i in range(num_layers):
            self.blks.add_module("block" + str(i),
                                 EncoderBlock(num_hiddens, multi_hiddens, num_heads,
                                              ffn_num_input, ffn_num_hiddens, norm1, norm2, dropout))
        '''测试encoder
        self.encoder = EncoderBlock(num_hiddens,multi_hiddens,num_heads,
                 ffn_num_input,ffn_num_hiddens,norm1,norm2,dropout)
        '''
        self.all_in_one = nn.Linear(max_len * num_hiddens, number_of_categories, bias=True)  # 用bias

    def forward(self, train_x):
#         train_x = self.pos_encoding(self.embedding(train_x) * torch.sqrt(torch.Tensor(self.num_hiddens)).cuda())  # 因为位置编码值在-1和1之间，
        train_x = self.pos_encoding(self.embedding(train_x) * math.sqrt(self.num_hiddens))
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        # 以上完成了位置编码
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            train_x = blk(train_x)
            self.attention_weights[
                i] = blk.attention.attention.attention_weights
        '''
        train_x = self.encoder(train_x)
        '''
        # 把（batch，序列长度，信息维度）直接降维到（batch，1） 用全连接层，或者是全局平均池化
        # train_x = torch.nn.functional.adaptive_avg_pool2d(train_x, (1,1)) #全局平均池化
        # train_x = train_x.reshape(-1,1)
        """全连接层"""
        train_x = train_x.reshape(-1, self.max_len * self.num_hiddens)
        train_x = self.all_in_one(train_x)
        return train_x

In [4]:
class DotProductAttention(nn.Module):
    """缩放点积注意力"""

    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    def forward(self, queries, keys, values):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights = nn.functional.softmax(scores, dim=-1)
        return torch.bmm(self.dropout(self.attention_weights), values)


class MultiHeadAttention(nn.Module):
    """多头注意力"""

    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=True, **kwargs):  # 没有用bias
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads)
        output = self.attention(queries, keys, values)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)


def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)


class PositionWiseFFN(nn.Module):  # 前馈网络完成
    """基于位置的前馈网络"""

    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))


# 归一化完成
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""

    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)


class EncoderBlock(nn.Module):
    def __init__(self, num_hiddens, multi_hiddens, num_heads,
                 ffn_num_input, ffn_num_hiddens, norm1, norm2, dropout, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,  # 定义多头注意力
                                            multi_hiddens, num_heads, dropout)

        self.addnorm1 = AddNorm(norm1, dropout)  # 定义归一化层 （110，16）

        self.ffn = PositionWiseFFN(  # 定义前馈网络
            ffn_num_input, ffn_num_hiddens, num_hiddens)

        self.addnorm2 = AddNorm(norm2, dropout)  # 定义归一化层 （110，16）

    def forward(self, train_x):
        cache_1 = self.addnorm1(train_x, self.attention(train_x, train_x, train_x))
        train_x = self.addnorm2(cache_1, self.ffn(cache_1))
        return train_x


In [5]:
class MyDataSet(data.Dataset):
    def __init__(self, train_x, train_y):
        super(MyDataSet, self).__init__()
        self.train_x = train_x
        self.train_y = train_y

    def __len__(self):
        return self.train_y.shape[0]

    def __getitem__(self, idx):
        return self.train_x[idx], self.train_y[idx]


In [39]:
def acc_f1(x,y,number_of_categories):
    """测试设置"""
    with torch.no_grad(): 
        
        net.eval()
        
        """与net.train对应"""
        tp=fn=fp=tn= 0
        #定义一个表，（actual，predicted）
        table=[[0 for i in range(number_of_categories)]for i in range(number_of_categories)]
        true = 0
        false = 0
        for length in range(x.shape[0]):
            x_hat = net(x[length].reshape(1,110))
            x_hat = x_hat.reshape(number_of_categories)
            table[y[length, 0]][torch.argmax(x_hat, dim=0)] +=1
        #         x_hat = x_hat.detach().cpu().numpy()
            if torch.argmax(x_hat, dim=0) == y[length, 0]:
                true += 1
            #计算tpfnfptn,F1-score微平均(micro-average)
        table = np.array(table)
        fp = table.sum(axis=0) - np.diag(table)
        fp = fp.sum()
        fn = table.sum(axis=1) - np.diag(table)
        fn = fn.sum()
        tp = np.diag(table)
        tp = tp.sum()
        tn = table.sum()*number_of_categories - (fp + fn + tp)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_out=(2*precision*recall)/(precision+recall)
        acc = true / x.shape[0]
    return acc,f1_out

def acc_epoch_f1(x,y,epoch,number_of_categories):
    """测试设置"""
    with torch.no_grad():
        
        net.eval()
        
        """与net.train对应"""
        tp=fn=fp=tn= 0
        #定义一个表，（actual，predicted）
        table=[[0 for i in range(number_of_categories)]for i in range(number_of_categories)]
        true = 0
        false = 0
        loss_sum = 0
        for length in range(x.shape[0]):
            x_hat = net(x[length].reshape(1,110))
            y_hat=y[length].reshape(1,1)
            loss_sum += criterion(x_hat,y_hat.view(-1)).item()
            x_hat = x_hat.reshape(number_of_categories)
        #         x_hat = x_hat.detach().cpu().numpy()
            table[y[length, 0]][torch.argmax(x_hat, dim=0)] +=1
            if torch.argmax(x_hat, dim=0) == y[length, 0]:
                true += 1
        cache = 'Epoch: {} and loss ={}'.format(epoch+1,loss_sum)
        write_file(output_file_val_loss,cache)
        #计算tpfnfptn,F1-score微平均(micro-average)
        table = np.array(table)
        fp = table.sum(axis=0) - np.diag(table)
        fp = fp.sum()
        fn = table.sum(axis=1) - np.diag(table)
        fn = fn.sum()
        tp = np.diag(table)
        tp = tp.sum()
        tn = table.sum()*number_of_categories - (fp + fn + tp)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_out=(2*precision*recall)/(precision+recall)
        acc = true / x.shape[0]
    return acc,f1_out


def f1(x,y,number_of_categories):
    """测试设置"""
    with torch.no_grad():
        
        net.eval()
        """与net.train对应"""

        tp=fn=fp=tn= 0
    
    #定义一个表，（actual，predicted）
        table=[[0 for i in range(number_of_categories)]for i in range(number_of_categories)]
    
        for length in range(x.shape[0]):
            x_hat = net(x[length].reshape(1,110))
            x_hat = x_hat.reshape(number_of_categories)
            table[y[length, 0]][torch.argmax(x_hat, dim=0)] +=1
    
    #计算tpfnfptn,F1-score微平均(micro-average)
        table = np.array(table)
        fp = table.sum(axis=0) - np.diag(table)
        fp = fp.sum()
        fn = table.sum(axis=1) - np.diag(table)
        fn = fn.sum()
        tp = np.diag(table)
        tp = tp.sum()
        tn = table.sum()*number_of_categories - (fp + fn + tp)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_out=(2*precision*recall)/(precision+recall)
    return f1_out

def f1_2(x,y,number_of_categories):
    """测试设置"""
    with torch.no_grad():
        
        net.eval()
        """与net.train对应"""

        tp=fn=fp=tn= 0
    
    #定义一个表，（actual，predicted）
        table=[[0 for i in range(number_of_categories)]for i in range(number_of_categories)]
    
        for length in range(x.shape[0]):
            x_hat = net(x[length].reshape(1,110))
            x_hat = x_hat.reshape(number_of_categories)
            table[y[length, 0]][torch.argmax(x_hat, dim=0)] +=1
    
    #计算tpfnfptn,F1-score微平均(micro-average)
        table = np.array(table)
        fp = table[1][0]
        fn = table[0][1]
        tp = table[0][0]
        tn = table[1][1]
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_out=(2*precision*recall)/(precision+recall)
    return f1_out

def write_file(filename,file_input):
    file1 = open(filename, 'a')
    file1.write(file_input+'\n')
    file1.close()
    return 0
def from18to5(y):
    list_y = []
    for i in range(len(y)):
        if y[i]<=9:
            list_y.append('0')
        if 9<y[i]<=10:
            list_y.append('1')
        if 10<y[i]<=11:
            list_y.append('2')
        if 11<y[i]<=13:
            list_y.append('3')
        if y[i]>13:
            list_y.append('4')
    out_y = np.array(list_y)
    out_y = out_y.reshape(len(out_y), 1)
    out_y = out_y.astype(np.float64)
    return out_y
def from18to2(y):
    list_y = [] 
    for i in range(len(y)):
        if y[i]<=11:
            list_y.append('0')
        if y[i]>11:
            list_y.append('1')
    out_y = np.array(list_y)
    out_y = out_y.reshape(len(out_y), 1)
    out_y = out_y.astype(np.float64)
    return out_y
def predict(test_x,seq,out_file_name,out_file_weight):
    torch.save(net.state_dict(), out_file_weight)
    """测试设置"""
    with torch.no_grad():
        
        net.eval()
        
        """与net.train对应"""
        y=np.zeros([test_x.shape[0],3])
        cache_y = []
        file = open(out_file_name, 'a')
        for length in range(test_x.shape[0]):
            x_hat = net(test_x[length].reshape(1,110))
            x_hat = x_hat.reshape(number_of_categories)
            y[length, 1] = torch.argmax(x_hat, dim=0)
            y[length, 2] = x_hat[torch.argmax(x_hat, dim=0)]
        for i in range(number_of_categories):
            max = 0
            for len_y in range(y.shape[0]):
                if y[len_y, 1] == i:
                    if y[len_y, 2] > max:
                        max = y[len_y, 2]
            cache_y.append(max)
        for leng in range(test_x.shape[0]):
            y[leng, 0] = y[leng, 1] + y[leng, 2]/10**(len(str(int(cache_y[int(y[leng, 1])]))))
            file.write(seq[leng]+'\t')
            file.write(str(y[leng, 0])+'\n')
        file.close()
    return 0

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 载入文件数据并处理为矩阵形式
# train_sequences = "../data/all_long_train.txt"
# test_sequences = "../data/all_long_have10000.txt"
# vaildation_sequences = "../data/all_long_no10000.txt"

# test_sequences = "../data/all_long_110_test.txt"

train_sequences = "../data/all_long_110_train.txt"
test_sequences = "../data/test_sequences.txt"
vaildation_sequences = "../data/all_long_110_val.txt"


"""载入训练集数据"""
train_x_init, train_y_init = process_data_110_withoutN(train_sequences)
# 训练集大小为(序列的数量，序列的长度)
# 由于训练集序列长度也是110，只有四万个序列超过了110.本次训练将超过110小于105的长度抛弃。
"""载入验证集数据"""
vaildation_x_init, vaildation_y_init = process_data_110_withoutN(vaildation_sequences)
"""载入测试集数据"""
test_x_init, test_y_init, seq = process_data_test(test_sequences)

#减少箱子数量
# train_y_init = from18to2(train_y_init)
# test_y_init = from18to2(test_y_init)
# vaildation_y_init = from18to2(vaildation_y_init)


# 处理数据
train_x_init = torch.from_numpy(train_x_init).to(device)  # 数据转换到tensor
train_y_init = torch.from_numpy(train_y_init).to(device)

test_x_init = torch.from_numpy(test_x_init).to(device)   # 数据转换到tensor
test_y_init = torch.from_numpy(test_y_init).to(device)

vaildation_x_init = torch.from_numpy(vaildation_x_init).to(device)
vaildation_y_init = torch.from_numpy(vaildation_y_init).to(device)

In [40]:
#数据转换
train_x = train_x_init.long()  # 数据转换到long
train_y = train_y_init.long()  # 数据转换到long 只保留了整数,实现18个箱子，后期可以根据精准度分为180，1800个箱子

test_x = test_x_init.long() # 数据转换到long
test_y = test_y_init.long()  # 数据转换到long 只保留了整数,实现18个箱子，后期可以根据精准度分为180，1800个箱子

vaildation_x = vaildation_x_init.long() # 数据转换到long
vaildation_y = vaildation_y_init.long()

sequence_type = 5  # 序列词汇种类
num_hiddens = 32  # 字 Embedding 的维度32
max_len = 110  # 序列最长长度
dropout = 0.1  # 设置dropout为0.1
multi_hiddens = 32  # 多头注意力的隐藏层，也是输出维度32
num_heads = 8  # 多头注意力设置4个头
ffn_num_input = 32  # 前馈网络第三维度32
ffn_num_hiddens = 64  # 隐藏层64
norm1 = [110, 32]  # 第一个归一化维度32
norm2 = [110, 32]  # 第二个归一化维度32
num_layers = 2  # 定义encoder的数量
batch_size = 2048  # 一次训练样本数量
number_of_categories = 18

output_file_train_loss = './project1_train_loss.txt'
output_file_val_loss = './project1_val_loss.txt'
output_file_vaildation_acc = './project1_vaildation_acc.txt'
output_file_test_acc = './project1_test_acc.txt'
output_file_time = './project1_time.txt'
output_file_vaildation_f1 = './project1_vaildation_f1.txt'
output_file_test_f1 = './project1_test_f1.txt'
output_file_weight = 'project1_weight.pth'
output_file_test = 'project1_test.txt'



# 上测试集查看准确率

loader = data.DataLoader(MyDataSet(train_x, train_y), batch_size, shuffle=True)
#   num_workers=8,第四个参数 ： 多线程读数据

# 定义网络
net = Net(num_hiddens, sequence_type, max_len, multi_hiddens, num_heads,
          num_layers, ffn_num_input, ffn_num_hiddens, norm1, norm2, dropout,number_of_categories).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# 开始训练
for epoch in range(1000000):
    start = time.time()
    """与acc测试对应"""
    net.train()
    """以上"""
    loss_sum = 0
    
    for train_x, train_y in loader:
        train_x = net(train_x)
        loss = criterion(train_x, train_y.view(-1))
        loss_sum+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    cache1 = 'Epoch: {} and loss ={}'.format(epoch+1,loss_sum)
    write_file(output_file_train_loss,cache1)
    
    vaildation_acc,vaildation__fl = acc_epoch_f1(vaildation_x,vaildation_y,epoch,number_of_categories)
#     test_acc,test__fl = acc_f1(test_x,test_y,number_of_categories)
    
    cache2="Epoch: {} and vaildation_acc: {}".format(epoch + 1,vaildation_acc)
    write_file(output_file_vaildation_acc,cache2)
#     cache3="Epoch: {} and test_acc is is: {}".format(epoch + 1,test_acc)
#     write_file(output_file_test_acc,cache3)
    cache4="Epoch: {} and vaildation__fl: {}".format(epoch + 1,vaildation__fl)
    write_file(output_file_vaildation_f1,cache4)
#     cache5="Epoch: {} and test__fl is is: {}".format(epoch + 1,test__fl)
#     write_file(output_file_test_f1,cache5)
    if (epoch % 40 == 0):
        epo=str(epoch)
        predict(test_x,seq,epo+output_file_test,epo+output_file_weight)
    end = time.time()
    cache='epoch {} took {} seconds'.format(epoch +1, end - start)
    write_file(output_file_time,cache)

KeyboardInterrupt: 